In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import docker
import logging
import time
import concurrent.futures
from datetime import datetime

In [10]:
# Discover the hardware architecture.
avail_cores = os.cpu_count()
print(f"Available cores: {avail_cores}")

Available cores: 48


In [ ]:
# Run CO-Located benchmarks in a Docker container on the same core.
def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True
    )
    print(f"Container started on CPU {task['cpuset_cpus']} for command: {task['command']}")

    # Capture container runtime
    # container_start_time = datetime.strptime(container.attrs['State']['StartedAt'], "%Y-%m-%dT%H:%M:%S.%fZ")
    container_start_time = container.attrs['State']['StartedAt']
    print(f"Container started at {container_start_time}")
    container.stop()
    # container_stop_time = datetime.strptime(container.attrs['State']['FinishedAt'], "%Y-%m-%dT%H:%M:%S.%fZ")
    # print(f"Container stopped at {container_stop_time}")
    # container_lifetime = (container_stop_time - container_start_time).total_seconds()
    # print(f"Container lifetime: {container_lifetime} seconds")
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    client = docker.from_env()

    # List of dictionaries to hold task information.
    tasks = [
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "2"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "--file-total-size=50G", "--file-test-mode=rndrw", "--num-threads=1", "prepare"], "cpuset_cpus": "3"}, 
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "cleanup"], "cpuset_cpus": "3"}
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Colocator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    
    for future in futures:
        future.result()
    print("All isolated benchmarks completed.")
    # Clean up Docker client.
    client.close()

In [6]:
# Run CO-Located benchmarks in a Docker container on the same core.
def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True
    )
    print(f"Container started on CPU {task['cpuset_cpus']} for command: {task['command']}")
    container.stop()
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    client = docker.from_env()
    
    print("Container started for Disk benchmark (prepare).")

    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=150G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )
    print(disk_prepare_output.decode("utf-8"))  


    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=20000000", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"}
        ]},
        {"MemFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "1"}
        ]},
        {"FileIOCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "1"}
        ]},
        {"CpuCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
        ]},
        {"MemMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"},
        ]},
        {"FileIOFileIO": [
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "1"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print(disk_cleanup_output.decode("utf-8"))  
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
        # Clean up Docker client
    client.close() 

Container started for Disk benchmark (prepare).
sysbench 1.0.20 (using system LuaJIT 2.1.0-beta3)

128 files, 1228800Kb each, 153600Mb total
Creating files for the test...
Extra file open flags: (none)
Creating file test_file.0
Creating file test_file.1
Creating file test_file.2
Creating file test_file.3
Creating file test_file.4
Creating file test_file.5
Creating file test_file.6
Creating file test_file.7
Creating file test_file.8
Creating file test_file.9
Creating file test_file.10
Creating file test_file.11
Creating file test_file.12
Creating file test_file.13
Creating file test_file.14
Creating file test_file.15
Creating file test_file.16
Creating file test_file.17
Creating file test_file.18
Creating file test_file.19
Creating file test_file.20
Creating file test_file.21
Creating file test_file.22
Creating file test_file.23
Creating file test_file.24
Creating file test_file.25
Creating file test_file.26
Creating file test_file.27
Creating file test_file.28
Creating file test_file.2

In [ ]:
# Construct affinity score matrix.